# Example: Simplify and Redensify a Watershed



In [1]:
# conda package imports
import numpy as np
from matplotlib import pyplot as plt
import logging
import copy
import shapely

import watershed_workflow 
import watershed_workflow.source_list
import watershed_workflow.ui
import watershed_workflow.crs
import watershed_workflow.densification
watershed_workflow.ui.setup_logging(1,None)

import ipympl

In [2]:
## Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed. 
coweeta_shapefile = '../../Coweeta_data/input_data/coweeta_basin.shp'
hint = '0601'  # hint: HUC 4 containing this shape.  
               # This is necessary to avoid downloading all HUCs to search for this shape
name = 'Coweeta'

figsize = (6,6)
figsize_3d = (8,6)


In [3]:
# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.daymet_crs()

### Helper Functions

In [4]:
from shapely.geometry import LineString
def trim_linestring(linestring, fraction, from_up=True):
    total_length = linestring.length
    trim_length = total_length * fraction

    if from_up:
        # Trimming from the start
        current_length = 0
        for i, point in enumerate(linestring.coords[:-1]):
            segment = LineString([point, linestring.coords[i + 1]])
            current_length += segment.length
            if current_length >= trim_length:
                return LineString([segment.interpolate(current_length - trim_length)] + linestring.coords[i + 1:])

    else:
        # Trimming from the end (original method)
        trimmed_length = total_length - trim_length
        current_length = 0
        for i, point in enumerate(linestring.coords[:-1]):
            segment = LineString([point, linestring.coords[i + 1]])
            current_length += segment.length
            if current_length >= trimmed_length:
                return LineString(linestring.coords[:i + 1] + [segment.interpolate(trimmed_length - (current_length - segment.length))])

    return linestring


### Sources and setup

In [ ]:
# set up a dictionary of source objects
sources = watershed_workflow.source_list.get_default_sources()
sources['hydrography'] = watershed_workflow.source_list.hydrography_sources['NHD Plus']
sources['HUC'] = watershed_workflow.source_list.huc_sources['NHD Plus']

watershed_workflow.source_list.log_sources(sources)

### Get HUCs

In [ ]:
# load hucs from shape
_, watershed = watershed_workflow.get_split_form_shapes(coweeta_shapefile, out_crs=crs)

### Get Rivers 

In [7]:
import pickle
with open("./data/coweeta_rivers_pickle", 'rb') as handle:
    rivers = pickle.load(handle)


In [ ]:
# quick look at rivers and watershed boundary
fig = plt.figure(figsize=(4,4))
ax = watershed_workflow.plot.get_ax(crs, fig, window=[0.05,0.1,0.9,0.8])
watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
watershed_workflow.plot.rivers(rivers, crs, 'b', ax)

plt.show()

### Simplify

In [ ]:
simplify = 60 # length scale to target average edge, ; set this as 1 or 2 when using new densify 
snap_tol = 40
# keeping the originals
rivers_orig=[river.deepcopy() for river in rivers]
watershed_orig=copy.deepcopy(watershed) 

# simplifying 
rivers = watershed_workflow.simplify(watershed, rivers, simplify_hucs=simplify, simplify_rivers=simplify,
                            snap_tol=snap_tol, cut_intersections=True)

# for plotting purpose only
rivers_simplified=[river.deepcopy() for river in rivers] 
watershed_simplified=copy.deepcopy(watershed) 

This error is because a reach has two intersections with the watershed boundary, a case of inconsistent data of mapped flowlines for river and watershed boundary. As a remedy, we could either trim the reach or move the river or watershed boundary segment. 

### 1) Trim the reach

We can get the problematic node using either node ID (if the dataset is from NHDPlus, this is NHD_ID).

In [10]:
# revert to original river
rivers = [river.deepcopy() for river in rivers_orig]

In [ ]:
## get the problematic node
node_id = '25000400108019'
node = rivers[0].getNode(node_id)
node.segment

Plotting original and simplified-pruned rivers and watershed


In [14]:
trimmed_reach_segment = trim_linestring(node.segment, 0.25, from_up=True) # Trims from upstream

In [ ]:
# let's have a loot at trimmed node
# quick look at rivers and watershed boundary
fig = plt.figure(figsize=(4,4))
ax = watershed_workflow.plot.get_ax(crs, fig, window=[0.05,0.1,0.9,0.8])
watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
watershed_workflow.plot.rivers(rivers, crs, 'b', ax)
watershed_workflow.plot.shply(trimmed_reach_segment, crs, 'r', ax)
plt.show()

In [16]:
# assign the edited reach segment back to the node
node.segment = trimmed_reach_segment

In [ ]:
# quick look at rivers and watershed boundary after editing river
fig = plt.figure(figsize=(4,4))
ax = watershed_workflow.plot.get_ax(crs, fig, window=[0.05,0.1,0.9,0.8])
watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
watershed_workflow.plot.rivers(rivers, crs, 'b', ax)
plt.show()

In [ ]:
# simplifying 
rivers = watershed_workflow.simplify(watershed, rivers, simplify_hucs=simplify, simplify_rivers=simplify,
                            snap_tol=0.75*simplify, cut_intersections=True)

# for plotting purpose only
rivers_simplified=[river.deepcopy() for river in rivers] 
watershed_simplified=copy.deepcopy(watershed) 

In [ ]:
rivers[0].is_continuous()

### 2) Remove the reach

We can get the problematic node using either node ID (if the dataset is from NHDPlus, this is NHD_ID).

In [20]:
# revert to original river
rivers = [river.deepcopy() for river in rivers_orig]

In [ ]:
## get the problematic node
node_id = '25000400108019'
node = rivers[0].getNode(node_id)
node.segment

In [ ]:
# remove the problematic node
node.remove()

In [ ]:
# quick look at rivers to see if the problematic node is removed
fig = plt.figure(figsize=(4,4))
ax = watershed_workflow.plot.get_ax(crs, fig, window=[0.05,0.1,0.9,0.8])
watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
watershed_workflow.plot.rivers(rivers, crs, 'b', ax)
plt.show()

In [ ]:
rivers[0].is_continuous()

### 3) Nudge the reach

We can get the problematic node using either node ID (if the dataset is from NHDPlus, this is NHD_ID).

In [25]:
# revert to original river
rivers = [river.deepcopy() for river in rivers_orig]

In [ ]:
## get the problematic node
node_id = '25000400108019'
node = rivers[0].getNode(node_id)
node.segment

In [ ]:
reach_backup = node.segment

# move some coordinates
for i in range(0,50):
    node.moveCoordinate(i, np.array(node.segment.coords[i]) + np.array([-320*(49-i)/50,0]))

# plot
fig = plt.figure(figsize=(8,8))
ax = watershed_workflow.plot.get_ax(crs, fig, window=[0.05,0.1,0.9,0.8])
watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
watershed_workflow.plot.rivers(rivers, crs, 'b', ax)
watershed_workflow.plot.shply(reach_backup, crs, 'r', ax, marker='x', markersize=2)
plt.show()


# # revert that change!
# river.segment = reach_backup

In [ ]:
# simplifying 
rivers = watershed_workflow.simplify(watershed, rivers, simplify_hucs=simplify, simplify_rivers=simplify,
                            snap_tol=0.75*simplify, cut_intersections=True)

# for plotting purpose only
rivers_simplified=[river.deepcopy() for river in rivers] 
watershed_simplified=copy.deepcopy(watershed) 

In [ ]:
rivers[0].is_continuous()